In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [5]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [6]:
json_election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_election_file.append(json.loads(line))

In [7]:
dict_tweets={}
i=0
for tweet in json_election_file:
    dict_tweets[i]=tweet
    i+=1
tweets = pd.DataFrame.from_dict(dict_tweets, orient='index')
tweets.reset_index(level=0, inplace=False)
tweets.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,withheld_in_countries
0,Sat Nov 28 18:54:46 +0000 2020,1332759857930383367,1332759857930383367,RT @ShadowingTrump: 3/3 brief P &amp; VP month...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,1606589686268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Nov 28 18:54:46 +0000 2020,1332759857678659584,1332759857678659584,@hudsont1728 @GmanisQ @joshdcaplan @lsferguson...,"<a href=""http://twitter.com/#!/download/ipad"" ...",True,1.332741e+18,1332741477341011970,1.271583e+18,1271583317725458433,...,1606589686208,"[47, 140]",{'full_text': '@hudsont1728 @GmanisQ @joshdcap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Nov 28 18:54:46 +0000 2020,1332759857884237839,1332759857884237839,@jamescgibson I think Biden and he team kinda ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.332759e+18,1332759496339419142,8.921513e+08,892151258,...,1606589686257,"[14, 140]",{'full_text': '@jamescgibson I think Biden and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sat Nov 28 18:54:46 +0000 2020,1332759857552908289,1332759857552908289,RT @CarterPEstes: My latest:\n\nhttps://t.co/i...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,1606589686178,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
4,Sat Nov 28 18:54:46 +0000 2020,1332759858018455553,1332759858018455553,RT @KelemenCari: President Trump is right. The...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,1606589686289,NaN,NaN,NaN,1.332597e+18,1332597045165744128,{'created_at': 'Sat Nov 28 08:07:48 +0000 2020...,"{'url': 'https://t.co/1QBCXzuTjP', 'expanded':...",NaN,NaN
5,Sat Nov 28 18:54:46 +0000 2020,1332759858265923600,1332759858265923600,RT @LLinWood: Facts are stubborn things.,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,1606589686348,NaN,NaN,NaN,1.332740e+18,1332740170840502272,{'created_at': 'Sat Nov 28 17:36:32 +0000 2020...,"{'url': 'https://t.co/f13PrLos9S', 'expanded':...",NaN,NaN
6,Sat Nov 28 18:54:46 +0000 2020,1332759858404319236,1332759858404319236,"RT @Steve_Leave: @thehill Don't do it Donald, ...","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,1606589686381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sat Nov 28 18:54:46 +0000 2020,1332759858463051777,1332759858463051777,RT @RT_com: President Donald Trump claimed he ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,1606589686395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sat Nov 28 18:54:46 +0000 2020,1332759858542759938,1332759858542759938,"RT @CopingMAGA: Trump really blew $3,000,000 t...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,1606589686414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Sat Nov 28 18:54:46 +0000 2020,1332759858588872704,1332759858588872704,RT @realDonaldTrump: Alan Dershowitz: Trump wa...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,1606589686425,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.set_option('display.max_columns', None)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 501 non-null    object 
 1   id                         501 non-null    int64  
 2   id_str                     501 non-null    object 
 3   text                       501 non-null    object 
 4   source                     501 non-null    object 
 5   truncated                  501 non-null    bool   
 6   in_reply_to_status_id      87 non-null     float64
 7   in_reply_to_status_id_str  87 non-null     object 
 8   in_reply_to_user_id        87 non-null     float64
 9   in_reply_to_user_id_str    87 non-null     object 
 10  in_reply_to_screen_name    87 non-null     object 
 11  user                       501 non-null    object 
 12  geo                        0 non-null      object 
 13  coordinates                0 non-null      object 

In [9]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)

In [10]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [11]:
tweets=remove_unnecessary_columns(tweets)

In [37]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=data['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                if word not in inv_index[word]:
                    inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

#REMOVE THIS FUNCTION, THERE MUST BE A LIBRARY THAT DOES THIS
def tf_idf_normalization(tf_idf: list) -> None:
    D_square=np.zeros(len(tf_idf))
    for i in range(len(tf_idf)):
        document_values=list(tf_idf[i].values())
        for j in range(len(document_values)):
            document_values[j]=(document_values[j]**2)
        D_square[i] = np.sum(document_values)
    
    for i in range (len(tf_idf)):
        for word in tf_idf[i]:
            tf_idf[i][word]= tf_idf[i][word]/np.sqrt(D_square[i])

def tf_idf(docs: list, inv_index: dict) -> list:
    tf=[]
    df={}
    idf={}
    tf_idf=[]
    N=len(tweets)
    for doc_text in docs:
        tf_doc_i={}
        words = doc_text.split()
        for word in words:
            if word in tf_doc_i:
                tf_doc_i[word]+=1/len(words)
            else:
                tf_doc_i[word]=1/len(words)
        tf.append(tf_doc_i)
        
    for word in inv_index:
        df[word]=len(inv_index[word])
        idf[word]=np.log(float(N/(df[word]+1)))

    for doc in range(len(docs)):
        tf_idf_per_doc={}
        for word in tf[doc]:
            tf_idf_per_doc[word]=tf[doc][word]*idf[word]
        tf_idf.append(tf_idf_per_doc)
        
    tf_idf_normalization(tf_idf)
    return tf_idf


In [75]:
def query_tfidf(query: str, query_vector: dict) -> None:
    query_tf_idf = tf_idf([query], inverted_index(tweets))
    for word in query.split():
        query_vector[word]= query_tf_idf[0][word]
    
def ranking_documents(query: str, tfidf: list):
    cosine_similarity={}
    i=0
    for document in tfidf:
        vectorized_doc = list(document.values())
        if all(word in document.keys() for word in query.split()):
            #THIS CAN BE IMPROVED
            vectorized_query = {key:0 for key in document.keys()}
            query_tfidf(query, vectorized_query)
            vectorized_query_values = list(vectorized_query.values())
            cosine_similarity[i] = np.dot(vectorized_query_values,vectorized_doc)
        else:
            cosine_similarity[i] = 0
        i+=1
    return cosine_similarity

In [81]:
database=get_tweet_data(tweets)
for i in range(len(tweets)):
    tweets['text'][i]=text_normalization(tweets['text'][i])

<ipython-input-81-9d9c5c666b60>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text'][i]=text_normalization(tweets['text'][i])


In [15]:
tweets["text"].tolist()
tfidf=tf_idf(tweets["text"].tolist(), inverted_index(tweets))

In [85]:
print("Insert your query:\n")
query = input()
query=text_normalization(query)
docs = ranking_documents(query, tfidf)    
top = 20

database["score"]= pd.DataFrame(np.zeros(len(database)))
database["score"]=docs.values()
database=database.sort_values(by=["score"], ascending=False)
database.head(top)

#CUBRIR LOS CASOS DONDE NO SE LLEGA AL TOP 20 BUSCANDO MAS TWEETS

Insert your query:

joe biden


,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,score
488,Alan Dershowitz: Trump was ‘Absolutely Right‘ ...,Donald J. Trump,Sat Nov 28 18:35:11 +0000 2020,None,24407,4988,https://twitter.com/i/web/status/1332759898849...,0.502910
165,On #SmallBusinessSaturday let’s always remembe...,Lauren Boebert,Sat Nov 28 18:34:44 +0000 2020,None,315,58,https://twitter.com/i/web/status/1332759871205...,0.353438
283,"And so people are waking up. We are at war, an...",Serenabit,Sat Nov 28 18:54:51 +0000 2020,None,0,0,https://twitter.com/i/web/status/1332759880780...,0.353438
80,"Actually, yes.\nIn 2012, Obama ran against a g...",Ana Navarro-Cárdenas,Sat Nov 28 16:22:56 +0000 2020,None,30465,3544,https://twitter.com/i/web/status/1332759864800...,0.324377
411,There are so many Biden-Harris signs around he...,TBS,Sat Nov 28 18:54:54 +0000 2020,None,0,0,https://twitter.com/i/web/status/1332759892009...,0.319600
489,"So, does the trump cult Kool-aid just wear off...",BrooklynDad_Defiant!,Sat Nov 28 06:58:51 +0000 2020,None,16770,1666,https://twitter.com/i/web/status/1332759899265...,0.235886
37,I'm an RN. It's baffling to me that the majori...,yada yada 🌊⚕♻️☮🐝🌎🏳️‍🌈,Fri Nov 27 23:45:08 +0000 2020,None,28495,2814,https://twitter.com/i/web/status/1332759860526...,0.233387
431,Computer fraud executed by foreign actors.,Lin Wood,Sat Nov 28 18:29:34 +0000 2020,None,4668,1874,https://twitter.com/i/web/status/1332759893992...,0.223772
342,Plus Communist China. CCP wants America for it...,Lin Wood,Sat Nov 28 18:17:28 +0000 2020,None,3429,1157,https://twitter.com/i/web/status/1332759885981...,0.223727
85,Here’s a look back at how Trump botched Americ...,The Daily Show,Sat Nov 28 14:01:00 +0000 2020,None,3784,1004,https://twitter.com/i/web/status/1332759864981...,0.221514


In [17]:
# get always full text by checking if truncated. If it is, then find extended_tweet or retweeted_status.

In [18]:
os.remove(OUTPUT_FILENAME)

In [19]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN